In [12]:
import pandas as pd
import pyproj
from pyproj import CRS, Transformer
from arcgis.gis import GIS
import os
import datetime
from pathlib import Path
from config import password, user
import zipfile

In [2]:
# User login info
gis = GIS('https://usys-ethz.maps.arcgis.com', user, password)

In [3]:
# Account and Hosted Feature Layer Name, with Feature Layer structure
username = 'B.Baeume'
groups = ['12', '34']
year = str(datetime.date.today().year)

In [4]:
# Define saving locations
folder = '04_Baeume_'
download = folder + year + '/Download'

In [7]:
# Output Path with file name
outputPath_download = os.path.join("/output")
Path(outputPath_download).mkdir(parents=True, exist_ok=True)
print(outputPath_download)

PermissionError: [Errno 13] Permission denied: '/output'

In [8]:
# Output Path with file name
outputPath_download = os.path.join("/home/jovyan", download)
Path(outputPath_download).mkdir(parents=True, exist_ok=True)
print(outputPath_download)


/home/jovyan/04_Baeume_2025/Download


In [9]:
# Download all data
def export_data(year, item, outputPath_download):
    downloadFormat = 'File Geodatabase'
    try:
        result = item.export(year + '_{}'.format(item.title), downloadFormat)
        result.download(r'{}'.format(outputPath_download))
        result.delete()
        print("Successfully downloaded " + item.title)
    except Exception as e:
        print(e)

In [13]:
# Download all data
def export_data(year, item, outputPath_download):
    downloadFormat = 'File Geodatabase'
    try:
        result = item.export(year + '_{}'.format(item.title), downloadFormat)
        result.download(r'{}'.format(outputPath_download))
        result.delete()
        print("Successfully downloaded " + item.title)
    except Exception as e:
        print(e)

def export_dataworkshop(allFeatureLayer, outputPath):
    # Define Input and Output Coordinate System
    inProj = CRS('epsg:3857')
    outProj = CRS('epsg:4326')
    transformer = Transformer.from_crs(inProj, outProj)

    # Counter for Group Name and empty Dataframe
    counter = 0
    dfAll = pd.DataFrame()

    # Loop through all Feature layers
    for featureLayer in allFeatureLayer:
        # Search all Features in a Feature Layer
        print(item)
        print(featureLayer.properties.name)
        featureSet = featureLayer.query()
        if not featureSet:
            counter += 1
            print('Feature Layer {} is empty'.format(featureLayer.properties.name))
            continue
        else:
            print('Feature Layer {} is being processed'.format(counter))
            # Create Panda Dataframe from all Features
            df = featureSet.sdf
            # Create empty X and Y List
            newX = []
            newY = []
            counter += 1
            # Convert each Coordinate to the new projection
            for f in featureSet:
                x1, y1 = f.geometry["x"], f.geometry["y"]
                x2, y2 = transformer.transform(x1, y1)
                newX.append(x2)
                newY.append(y2)

            # Insert new Column X and Y in Data Frame
            df.insert(len(df.columns), 'X', newY)
            df.insert(len(df.columns), 'Y', newX)

            # Insert Groupname
            df.insert(len(df.columns), 'Gruppe', '{}{}'.format(group, f"{counter:02d}"))
            print(df)

        # Append Dataframe
        dfAll = pd.concat([dfAll, df], ignore_index=True)

    # Debugging: Check if DataFrame is created correctly
    print("DataFrame created:")
    print(dfAll.head())

    # Write Dataframe to Excel
    try:
        dfAll.to_excel(outputPath, index=False)
        print(f"Excel file created at {outputPath}")
    except Exception as e:
        print(e)


In [14]:
for group in groups:
    print(group)
    # Run through the groups defined to download data from all layers
    title = 'Baeume_' + group
    print('Starting:')
    print(title)

    # Define and create the folder where the dataworkshop will be stored
    workshop_folder = folder + year + '/Datenworkshop'
    Path(workshop_folder).mkdir(parents=True, exist_ok=True)

    workshop = workshop_folder + "/" + title + ".xlsx"
    outputPath_workshop = os.path.join("/home/jovyan", workshop)

    print(outputPath_workshop)

    # Search hosted Feature Layer
    hostedFeatureLayer = gis.content.search('owner: {} AND title: {} AND type: Feature Service'.format(username, title))
    print(hostedFeatureLayer)

    # Store first Item of hostedFeature Search
    item = hostedFeatureLayer[0]
    print(item)
    export_data(year, item, outputPath_download)
    # Store all Feature Layers of the item
    allFeatureLayer = item.layers
    print('FeatureLayer')
    print(allFeatureLayer)
    export_dataworkshop(allFeatureLayer, outputPath_workshop)

    # Create a zip file of the downloaded data
    zip_file_path = os.path.join("/home/jovyan", f"{title}.zip")
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        for root, dirs, files in os.walk(outputPath_download):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), outputPath_download))

    print(f"Files are ready for download: {zip_file_path} and {outputPath_workshop}")

12
Starting:
Baeume_12
/home/jovyan/04_Baeume_2025/Datenworkshop/Baeume_12.xlsx
[<Item title:"Baeume_12" type:Feature Layer Collection owner:B.Baeume>]
<Item title:"Baeume_12" type:Feature Layer Collection owner:B.Baeume>
Successfully downloaded Baeume_12
FeatureLayer
[<FeatureLayer url:"https://services-eu1.arcgis.com/bW9OgbfdKtBIPrUj/arcgis/rest/services/Baeume_12/FeatureServer/0">]
<Item title:"Baeume_12" type:Feature Layer Collection owner:B.Baeume>
Baeume_12
Feature Layer 0 is being processed
   OBJECTID Artname Sicherheit Bemerkung  Durchmesser_BHD  ID_Bestand  \
0         1   Tanne       <NA>      <NA>               59           4   
1         2   Tanne       <NA>      <NA>               56           4   
2         3   Tanne       <NA>      <NA>               61           4   
3         4   Tanne       <NA>      <NA>               67           0   
4         5   Buche       <NA>      <NA>               62           0   
5         6   Tanne       <NA>      <NA>               66  

/srv/conda/envs/notebook/lib/python3.10/site-packages/arcgis/features/geo/_io/serviceops.py:141: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["SHAPE"].replace({np.nan: None}, inplace=True)


[<Item title:"Baeume_34" type:Feature Layer Collection owner:B.Baeume>]
<Item title:"Baeume_34" type:Feature Layer Collection owner:B.Baeume>
Successfully downloaded Baeume_34
FeatureLayer
[<FeatureLayer url:"https://services-eu1.arcgis.com/bW9OgbfdKtBIPrUj/arcgis/rest/services/Baeume_34/FeatureServer/0">]
<Item title:"Baeume_34" type:Feature Layer Collection owner:B.Baeume>
Baeume_34
Feature Layer 0 is being processed
   OBJECTID Artname Sicherheit Bemerkung  Durchmesser_BHD  ID_Bestand  \
0         1   Tanne       <NA>      <NA>               60           4   
1         2   Tanne       <NA>      <NA>             <NA>           4   
2         3   Tanne       <NA>      <NA>             <NA>           4   
3         4   Tanne       <NA>      <NA>             <NA>           0   
4         5   Buche       <NA>      <NA>             <NA>           0   
5         6   Tanne       <NA>      <NA>             <NA>           0   

  ETHUsername Datum                              GlobalID  \
0   

/srv/conda/envs/notebook/lib/python3.10/site-packages/arcgis/features/geo/_io/serviceops.py:141: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["SHAPE"].replace({np.nan: None}, inplace=True)
